# 朴素贝叶斯
- 优点
    - 在数据较少时仍旧有效，可以处理多类别问题
- 缺点
    - 对于输入数据的准备方式敏感
- 适用数值类型：标称性数据

贝叶斯决策论的核心思想就是选择具有高概率的决策

贝叶斯概率引入先验知识和逻辑推理来处理不确定命题，另一种概率解释称为frequency probability，它只从数据本身获取结论而不考虑推理及先验知识

贝叶斯公式帮助我们交换条件概率中的条件与结果
$$
p(c|x) = \frac{p(x|c)p(c)}{p(x)}
$$

# 使用NB进行文本分类
> 朴素贝叶斯假设每个特征同等重要且相互独立，这也正是其名字的由来，尽管这些假设并不合理，但朴素贝叶斯确实起到了不错的效果

- 从文本中获取特征需要拆分文本
- 此处特征是来自文本的词条（token），其可以是任意字符组合，可以理解为单词，也可以使用非单词词条如url，IP地址等等
- 还需要将每个文本片段表示为一个词向量，1表示出现，0表示未出现

以在线社区留言板为例，为了不影响社区发展，需要屏蔽侮辱性言论，因此需要构建一个快速过滤器，对此需要设置连个类别：侮辱和非侮辱，使用1和0表示


## 准备数据：从文本中构建词向量

In [1]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    # 1表示侮辱性词汇，0表示没有
    classVec = [0,1,0,1,0,1]
    return postingList,classVec

# 创建词汇列表
def createVocabList(dataSet):
    vocabSet = set([])  
    for document in dataSet:
        # 合并两个集合
        vocabSet = vocabSet | set(document) 
    return list(vocabSet)

def setOfWords2Vec(vocabList,inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print('the word: %s is not in Vocabulary!'%word)
    return returnVec

listOPosts,listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
print(myVocabList)

['flea', 'food', 'so', 'garbage', 'please', 'mr', 'buying', 'problems', 'licks', 'take', 'steak', 'my', 'not', 'ate', 'maybe', 'has', 'dog', 'stupid', 'cute', 'I', 'is', 'dalmation', 'love', 'how', 'posting', 'him', 'help', 'worthless', 'stop', 'quit', 'park', 'to']


In [2]:
print(setOfWords2Vec(myVocabList,listOPosts[0]))

[1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]


## 训练算法：从词向量计算概率
W表示词向量，c表示类别那么贝叶斯公式可以表示为$ p(c_i|w) = \frac{p(w|c_i)p(c_i)}{p(w)}$, 为此我们需要
1. 首先获得$p(c_i)$,通过计算每个类别在总文档中所占比例
2. 由于朴素贝叶斯假设$p(w|c_i) = p(w_0|c_i)p(w_1|c_i)p(w_2|c_i)\cdots p(w_n|c_i)$,由此可得$p(w|c_i)$

In [3]:
from numpy import *
# 朴素贝叶斯分类器训练函数
def trainNB0(trainData,TrainClass):
    numTrainDocs = len(trainData)
    numWords = len(trainData[0])
    proAbusive = sum(TrainClass)/float(numTrainDocs)
    
    pro0Num = zeros(numWords)
    pro1Num = zeros(numWords)
    pro0Denom = 0.0
    pro1Denom = 0.0
    
    for i in range(numTrainDocs):
        if TrainClass[i]==1:
            pro1Num += trainData[i]
            pro1Denom += sum(trainData[i])
        else:
            pro0Num += trainData[i]
            pro0Denom += sum(trainData[i])
    pro1Vec = pro1Num/pro1Denom
    pro0Vec = pro0Num/pro0Denom
    
    return pro0Vec,pro1Vec,proAbusive



In [4]:
listOPosts,listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList,postinDoc))

In [5]:
for i in range(len(trainMat)):
    print(trainMat[i])

[1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0]


In [6]:
p0V,p1V,pAb = trainNB0(trainMat,listClasses)

In [7]:
print(pAb)
print(p0V)
print(p1V)

0.5
[0.04166667 0.         0.04166667 0.         0.04166667 0.04166667
 0.         0.04166667 0.04166667 0.         0.04166667 0.125
 0.         0.04166667 0.         0.04166667 0.04166667 0.
 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667
 0.         0.08333333 0.04166667 0.         0.04166667 0.
 0.         0.04166667]
[0.         0.05263158 0.         0.05263158 0.         0.
 0.05263158 0.         0.         0.05263158 0.         0.
 0.05263158 0.         0.05263158 0.         0.10526316 0.15789474
 0.         0.         0.         0.         0.         0.
 0.05263158 0.05263158 0.         0.10526316 0.05263158 0.05263158
 0.05263158 0.05263158]


## 测试算法： 根据实际情况改进分类器
利用贝叶斯分类器对文档分类时，需要计算多个概率乘积获得某个文档分配的概率，即
$$p(w_0|c_0)p(w_1|c_0)\cdots p(w_n|c_0)$$
---
当其中一个概率为0时，最后乘积也是0，为了减低该影响，可以进行Laplace校准
$$
p_\lambda(X^{(j)}=a_{jI}|Y=c_k) = \frac{\sum^N_{i=1}I(x_i^{(j)} = a_{jI},y_i=c_k)+\lambda}{\sum^N_{i=1}I(y_i=c_k)+S_j\lambda}
$$
其中$a_jl$表示j个特征的第I个属性，$S_J$表示第j个特征的属性个数，k代表种类个数

---
此外由于每个$p(w_i|1)$都小于1，多个很小数字相乘会出现下溢，一种有效的解决方法是对小数取自然对数，$ln(a*b) = ln(a)+ln(b)$于是通过求对数可以避免下溢

$f(x)和ln(f(x))$**同增减**，且在相同点上取极值，虽然取值不同但不影响结果


In [8]:
def trainNB0(trainData,TrainClass):
    numTrainDocs = len(trainData)
    numWords = len(trainData[0])
    proAbusive = sum(TrainClass)/float(numTrainDocs)
    
    pro0Num = ones(numWords)
    pro1Num = ones(numWords)
    pro0Denom = 2.0
    pro1Denom = 2.0
    
    for i in range(numTrainDocs):
        if TrainClass[i]==1:
            pro1Num += trainData[i]
            pro1Denom += sum(trainData[i])
        else:
            pro0Num += trainData[i]
            pro0Denom += sum(trainData[i])
    pro1Vec = log(pro1Num/pro1Denom)
    pro0Vec = log(pro0Num/pro0Denom)
    
    return pro0Vec,pro1Vec,proAbusive

In [9]:
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    # 这里由于是对数形式因此相加等于相乘
    p1 = sum(vec2Classify*p1Vec) + log(pClass1)
    p0 = sum(vec2Classify*p0Vec) + log(1-pClass1)
    if p1>p0:
        return 1
    else:
        return 0
    
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love','my','dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,' classified as : ' ,classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid','garbage']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,' classified as : ' ,classifyNB(thisDoc,p0V,p1V,pAb))
    
testingNB()

['love', 'my', 'dalmation']  classified as :  0
['stupid', 'garbage']  classified as :  1


## 准备数据： 文档词袋模型
- 目前为止，我们将每个词是否出现作为特征，其被称为set-of-words model
- 如果一个词在文档出现不止一次，该方法不能体现该信息，这种方法被称为 bags-of-words model
- 在词袋模型中每个单词可以出现多次，词集中每个词只能出现一次

In [10]:
# 朴素贝叶斯词袋模型
def bagOfWords2VecMN(vocabList,inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [11]:
testEntry = ['love','my','dalmation','my']
print(bagOfWords2VecMN(myVocabList,testEntry))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# 使用NB过滤垃圾邮件
1. 收集数据：提供文本文件
2. 准备数据：将文本解析成词条向量
3. 分析数据：检查词条确保解析的正确性
4. 训练算法：使用之前的建立的trainNB0()函数
5. 测试算法：使用classifyNB()测试，并鬼剑测试函数计算文档的错误率
6. 使用算法：构建完整程序对文档进行分类

## 准备数据：切分文本，构建词汇表

In [12]:
import re
mySent = 'This book is the best book on Python or M.L. I have ever laid eyes upon.'
# 使用正则表达式清楚标点符号，不过有些情况下标点对于文本语义的表达也很重要
regEx = re.compile('\W+')
listofTokens = regEx.split(mySent)
print(listofTokens)

['This', 'book', 'is', 'the', 'best', 'book', 'on', 'Python', 'or', 'M', 'L', 'I', 'have', 'ever', 'laid', 'eyes', 'upon', '']


In [13]:
# 实际上句子查找，首字母大写很重要，但是对于此处的文本仅仅看做词袋，因此我们需要统一文本形式
print([tok.lower() for tok in listofTokens if len(tok)>0 ])

['this', 'book', 'is', 'the', 'best', 'book', 'on', 'python', 'or', 'm', 'l', 'i', 'have', 'ever', 'laid', 'eyes', 'upon']


In [14]:
emailText = open('../data/NaiveBayes/email/ham/6.txt').read()
listOfTokens = regEx.split(emailText)
print(listOfTokens)

['Hello', 'Since', 'you', 'are', 'an', 'owner', 'of', 'at', 'least', 'one', 'Google', 'Groups', 'group', 'that', 'uses', 'the', 'customized', 'welcome', 'message', 'pages', 'or', 'files', 'we', 'are', 'writing', 'to', 'inform', 'you', 'that', 'we', 'will', 'no', 'longer', 'be', 'supporting', 'these', 'features', 'starting', 'February', '2011', 'We', 'made', 'this', 'decision', 'so', 'that', 'we', 'can', 'focus', 'on', 'improving', 'the', 'core', 'functionalities', 'of', 'Google', 'Groups', 'mailing', 'lists', 'and', 'forum', 'discussions', 'Instead', 'of', 'these', 'features', 'we', 'encourage', 'you', 'to', 'use', 'products', 'that', 'are', 'designed', 'specifically', 'for', 'file', 'storage', 'and', 'page', 'creation', 'such', 'as', 'Google', 'Docs', 'and', 'Google', 'Sites', 'For', 'example', 'you', 'can', 'easily', 'create', 'your', 'pages', 'on', 'Google', 'Sites', 'and', 'share', 'the', 'site', 'http', 'www', 'google', 'com', 'support', 'sites', 'bin', 'answer', 'py', 'hl', 'en',

上述读取的是某公司告知不在进行某些支持的一封邮件，由于其中包含URL因此会出现一部分无意义单词（en,py等）,想去掉这些无意义单词，一种简单的方式是过滤长度小于3的字符串，本例通过使用一个通用的文本解析规则实现这一点

实际的解析程序中，要用更加高级的过滤器对HTML，URL对象进行处理，当前一个URL最后被解析成词汇表中的单词，文本解析是一个相当复杂的过程，在此没有进行深入的尝试

## 测试算法：NB进行交叉验证

In [15]:
# 文本解析
def textParse(bigString):
    listOfTokens = re.split(r'\W+',bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

def spamTest():
    docList = []
    classList = []
    fullText = []
    # 获得解析文本
    for i in range(1,26):
        wordList = textParse(open('../data/NaiveBayes/email/spam/%d.txt'%i).read())
        docList.append(wordList)
        fullText.append(wordList)
        classList.append(1)    
        wordList = textParse(open('../data/NaiveBayes/email/ham/%d.txt'%i).read())
        docList.append(wordList)
        fullText.append(wordList)
        classList.append(0)

    # 构造词汇表    
    vocabList = createVocabList(docList)

    # 随机构造训练，测试集
    trainingSetIndex = [x for x in range(50)] 
    testSet = []
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSetIndex)))
        testSet.append(trainingSetIndex[randIndex])
        del(trainingSetIndex[randIndex])

    # 构造训练集
    trainMat = []
    trainClasses = []
    for docIndex in trainingSetIndex:
        trainMat.append(bagOfWords2VecMN(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    # 构造模型
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))

    # 测试
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList,docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is : ',str(errorCount/len(testSet)))


In [16]:
spamTest()

the error rate is :  0.2


# 使用NB从个人广告中获取区域倾向

这个例子中，我们分别从美国两个城市选一批人，通过分析他们发布的征婚广告的信息来比较两个城市的人们在广告用词上是否不同。如果不同，他们各自常用的词是哪些？从人们用词中，考虑是否能对不同城市的人所关心的内容有所了解呢？

---

1. 收集数据：从RSS源收集内容，这里需要对RSS源构建一个接口
2. 准备数据：将文本文件解析成词条向量
3. 分析数据：检查词条确保解析正确
4. 训练算法：使用我们之前建立的trainNB0()函数
5. 测试算法：观察错误率，确保分类器可用。可以修改切分程序，降低错误率，提高分类结果
6. 使用算法：构建一个完整程序，封装代码，给定两个RSS源，显示常用公共词
---
使用来自不同城市的广告训练一个分类器，目的并非使用该分类器进行分类，而是通过观察单词和条件概率值来发现特定城市的相关的内容

## 获取数据

In [17]:
# 使用feedparser解析Rss数据
import feedparser
nasa = feedparser.parse('http://www.nasa.gov/rss/dyn/image_of_the_day.rss')

In [18]:
len(nasa['entries'])

60

In [19]:
import feedparser
history = feedparser.parse('https://www.sup.org/rss/?feed=history')

In [20]:
len(history['entries'])

15

In [21]:
import operator
def calcMostFreq(vocabList,fullText):
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token)
    sortedFreq = sorted(freqDict.items(),key = operator.itemgetter(1),reverse=True)
    return sortedFreq[:30]

In [22]:
import feedparser

def localWords(feed0,feed1):
    docList = []
    classList = []
    fullText = []

    minLen = min(len(feed1['entries']),len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
    vocabList = createVocabList(docList)

    # 去除出现次数最多的词
    top30Words = calcMostFreq(vocabList,fullText)
    for pairW in top30Words:
        if pairW[0] in vocabList:
            vocabList.remove(pairW[0])

    # 随机构造训练测试集
    trainingSetIndex = [x for x in range(2*minLen)] 
    testSet = []
    for i in range(20):
        randIndex = int(random.uniform(0,len(trainingSetIndex)))
        testSet.append(trainingSetIndex[randIndex])
        del(trainingSetIndex[randIndex])

    # 构造训练集
    trainMat = []
    trainClasses = []
    for docIndex in trainingSetIndex:
        trainMat.append(bagOfWords2VecMN(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])

    # 构造模型
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))

    # 测试
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList,docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is : ',str(errorCount/len(testSet)))
    
    return vocabList,p0V,p1V

In [23]:
vocabList,p0V,p1V = localWords(nasa,history)

the error rate is :  0.5


In [24]:
len(vocabList)

1457

**此处错误率远高于垃圾邮件中的错误率，但由于这里关注的是单词概率而不是实际分类，因此这个问题不重要**

## 分析数据：显示相关用词

In [32]:
vocabList,p0V,p1V = localWords(nasa,history)
topNASA = []
topHis = []
for i in range(len(p0V)):
    if p0V[i] > -3.5:
        topNASA.append((vocabList[i],p0V[i]))
    if p1V[i] > -5:
        topHis.append((vocabList[i],p1V[i]))
        
print('*'*10 + 'NASA' + '*'*10)
sortedNASA = sorted(topNASA,key = lambda x:x[1],reverse=True)
for item in sortedNASA:
    print(item[0] + ': ' +str(item[1]))
print('*'*10 + 'History' + '*'*10)
sortedHis = sorted(topHis,key = lambda x:x[1],reverse=True)
for item in sortedHis:
    print(item[0] + ': ' +str(item[1]))

the error rate is :  0.45
**********NASA**********
2021: -2.890371757896165
dec: -2.890371757896165
crew: -2.890371757896165
aboard: -3.1780538303479458
launch: -3.1780538303479458
nasa: -3.1780538303479458
rocket: -3.1780538303479458
station: -3.1780538303479458
spacex: -3.1780538303479458
image: -3.1780538303479458
**********History**********
iran: -4.785107951426769
benatar: -4.785107951426769
sex: -4.785107951426769
police: -4.939258631254028
work: -4.939258631254028
arani: -4.939258631254028
years: -4.939258631254028


# 总结
- 对于分类而言使用概率比使用硬规则更加有效，贝叶斯准则提供一直利用已知数值估计未知概率的有效方法
- 通过特征之间的条件独立性假设，降低对数据需求，当然这个假设过于简单因此被称为朴素贝叶斯。尽管该假设不正确，但其仍旧是一个有效的分类器
- 利用编程语言实现NB需要解决很多实际问题，比如下溢，其可以通过取对数来改进。词袋模型在解决文档分类比词集模型更有效，还可以通过移除停用词和对切分器优化改进性能
